In [8]:
import os
import subprocess
import numpy as np
from tifffile import imread,imwrite

In [2]:
input_dir = r'R:\Aidan\ULA_timecourse_ARC_006_temp'
output_dir = r'R:\Aidan\ULA_timecourse_ARC_006_movies'
os.makedirs(output_dir, exist_ok=True)

BIT_DEPTH = np.uint16
CREATE_SMALL_MOVIES = True
SMALL_FACTOR = 4
TIME_POINTS = 28

In [3]:
############################################################################
# run this to process all the positions
############################################################################

im_list = list(set([im.split('_')[0] for im in os.listdir(input_dir)]))
im_list

############################################################################
# uncomment and change im_list if you want to run only on selected images
############################################################################
#im_list = ['0944S']
#im_list

['0944S']

In [9]:
# make sure that alignment channel is set to the bright field images
# ex. '--align-channel', '1' if bright field is the second channel

for label in im_list:

    print(f'Processing {label}.')
    
    # Construct the file series arguments for all rounds
    fileseries_args = [f"fileseries|{input_dir}|pattern={label}_ch_{{channel:2}}_m_{{series:2}}_t_{str(round_num).zfill(3)}.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325" for round_num in range(TIME_POINTS)]

    # Create the full command as a list of arguments
    command = [
        'ashlar',
        *fileseries_args,
        '--output', f"{output_dir}\\{label}_ashlar_{{cycle:3}}_{{channel:2}}.tif",
        '--maximum-shift', '200',
        '--align-channel', '1',
    ]

    # Execute the command using subprocess
    subprocess.run(command)

    # #######################################################################################
    # # arrange into a stack
    # #######################################################################################
    
    # get channel list
    channel_list = list(set([x[-5] for x in os.listdir(output_dir) if f'{label}_ashlar' in x]))

    for ch in channel_list:
            
        frame_name_list = [x for x in os.listdir(output_dir) if ((f'{label}_ashlar' in x) and (f'{ch}.tif' in x))]
        frame_name_list.sort()

        frame_list = []
        for frame in frame_name_list:

            frame_path = os.path.join(output_dir,frame)
            im = imread(frame_path)

            frame_list.append(im)

        save_path = os.path.join(output_dir,f'{label}_ch_{ch}.tif')
        
        im_to_save = np.array(frame_list).astype(BIT_DEPTH)
        #imwrite(save_path, im_to_save, imagej=True)

        # create a small movie for quick inspection
        if CREATE_SMALL_MOVIES:
            save_path_small = save_path.replace('.tif','_small.tif')
            imwrite(save_path_small, im_to_save[:,::SMALL_FACTOR,::SMALL_FACTOR], imagej=True)

Processing 0944S.
